In [1]:
!pip install keras==2.1.5 --no-deps

     |████████████████████████████████| 337kB 2.7MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My Drive/google_colab_gpu/GSOC 2020/CERN-HSF

/content/drive/My Drive/google_colab_gpu/GSOC 2020/CERN-HSF


In [5]:
ls

cifar10vgg.h5          output.csv
ground_truth.csv       pytorch1.pt
Keras_CERN.ipynb       Pytorch_CERN.ipynb
Keras_inference.ipynb  SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
model1.h5              SingleElectronPt50_IMGCROPS_n249k_RHv1_inference.hdf5
model1.hdf5            SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5
model1.json            SinglePhotonPt50_IMGCROPS_n249k_RHv1_inference.hdf5
model2.h5


In [0]:
filename1='SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
filename2='SingleElectronPt50_IMGCROPS_n249k_RHv1_inference.hdf5'
data1 = h5py.File(filename1, 'r')
Y1=data1['y']
X1=data1['X']
write1=h5py.File(filename2, 'w')
write1.create_dataset('X', data=X1[:1000,:,:,:])
write1.create_dataset('y', data=Y1[:1000])
del(X1)
del(Y1)
data1.close()
write1.close()

In [0]:
filename1='SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
filename2='SinglePhotonPt50_IMGCROPS_n249k_RHv1_inference.hdf5'
data1 = h5py.File(filename1, 'r')
Y1=data1['y']
X1=data1['X']
write1=h5py.File(filename2, 'w')
write1.create_dataset('X', data=X1[:1000,:,:,:])
write1.create_dataset('y', data=Y1[:1000])
del(X1)
del(Y1)
data1.close()
write1.close()

In [6]:
import numpy as np
import pandas as pd
import h5py
#import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
#from keras.utils import np_utils
from keras.models import load_model,model_from_json
#from sklearn.model_selection import train_test_split
from keras import optimizers
from numpy import savetxt
from sklearn.utils import shuffle
print("Importing completed.")

Using TensorFlow backend.


Importing completed.


In [7]:
#Enter the batch size you want to evaluate on
batch_size=input("Enter the batch size(max 1000): ")

Enter the batch size(max 1000): 100


In [0]:
batch_start_idx=0
batch_end_idx=int(batch_size)

In [0]:
filename='SingleElectronPt50_IMGCROPS_n249k_RHv1_inference.hdf5'
data1 = h5py.File(filename, 'r')
Y1=data1['y']
X1=data1['X']

In [10]:
filename='SinglePhotonPt50_IMGCROPS_n249k_RHv1_inference.hdf5'
data0 = h5py.File(filename, 'r')
Y0=data0['y']
X0=data0['X']
print("Data loading completed.")

Data loading completed.


In [0]:
X_final=np.concatenate((X0[:],X1[:]),axis=0)
Y_final=np.concatenate((Y0[:],Y1[:]),axis=0)

In [0]:
X_final=(X_final[:,:,:,0].reshape((X_final.shape[0],X_final.shape[1],X_final.shape[2],1)))

In [0]:
X_final, Y_final = shuffle(X_final, Y_final)


In [14]:
data1.close()
data0.close()
print("Data preparation completed.")

Data preparation completed.


In [0]:
#model1=load_model('model1.hdf5')

In [0]:
#model1.save('model1.hdf5')

In [15]:
# load json and create model
json_file = open('model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model1 = model_from_json(loaded_model_json)
# load weights into new model
model1.load_weights("model1.hdf5")
print("Loaded model from disk")










Loaded model from disk


In [16]:
adam=optimizers.adam(lr=0.001)
model1.compile(loss = 'binary_crossentropy', optimizer=adam,metrics = [ 'accuracy','mae','mse'])
print("Model compiled successfully")



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model compiled successfully


In [0]:
target=model1.predict(X_final[batch_start_idx:batch_end_idx])
target[target<0.5]=0
target[target>0.5]=1
true_values=Y_final[batch_start_idx:batch_end_idx]
target=target.reshape(true_values.shape)

In [20]:
metrics=model1.evaluate(X_final[batch_start_idx:batch_end_idx,:,:,:],true_values.reshape((true_values.shape[0],1)),verbose=1)
print(model1.metrics_names[0]+": "+str(metrics[0])+"\n"+model1.metrics_names[1]+": "+str(metrics[1])+"\n"+model1.metrics_names[2]+": "+str(metrics[2])+"\n"+str(model1.metrics_names[3])+": "+str(metrics[3]))

100/100 [==============================] - 0s 414us/step
loss: 0.45229004859924316
acc: 0.74
mean_absolute_error: 0.3128185403347015
mean_squared_error: 0.14958342432975769


In [0]:
savetxt('output.csv',target,delimiter=',')
savetxt('ground_truth.csv',true_values,delimiter=',')

In [22]:
print("The outputs have been saved in file named output.csv corresponding to the respective indices. The first 20 values of output are:\n"+str(target[:20]))

The outputs have been saved in file named output.csv corresponding to the respective indices. The first 20 values of output are:
[0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1.]


In [23]:
print("The first 20 values of true values are:\n"+str(true_values[:20]))

The first 20 values of true values are:
[0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0.]
